In [6]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pyfolio as pf
import matplotlib
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [53]:
class TestStrategy(bt.Strategy):
    
    params = dict(profit_percent=0.3, loss_percent = 0.2, efast = 10, eslow=30)
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        tm = self.datas[0].datetime.time(0)
        print('%s, %s,  %s' % (dt.isoformat(), tm.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.time = self.datas[0].datetime.time
        self.order = None
        self.price = None
        self.comm = None
        emafast = bt.ind.EMA(period=self.p.efast)
        emaslow = bt.ind.EMA(period=self.p.eslow)
        self.crossover = bt.ind.CrossOver(emafast,emaslow)
        
    def notify_order(self, order):
        date = self.data.datetime.datetime().date()
        
        if order.status in [order.Submitted, order.Accepted]:
            return
            
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.price = order.executed.price
                self.comm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Margin, order.Rejected]:
            self.log('Order Margin/Rejected')
        
        elif order.status in [order.Canceled]:
            self.log('Order Canceled')
            #Closing Bracket orders comes under this condition

        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
                 
    def next(self):
        #self.log('Close, %.2f' % self.dataclose[0])

        if self.order:
            return
        
        #adding the 15 minute buffer
        if (self.data.datetime.time() < datetime.time(9,30)) or (self.data.datetime.time() > datetime.time(15,15)):
            return
        
        if not self.position:
            #Check the order while placing the amounts for different types, limit order and to do functions
            #like find a breakout and confirm the indicator. Many a times generates errors
            if self.crossover>0:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                entry = self.dataclose
                self.order = self.buy_bracket(limitprice=entry*(1+(self.p.profit_percent/100)),
                                              stopprice=entry*(1-(self.p.loss_percent/100)),
                                              exectype=bt.Order.Market)

        #Uncomment this and find that it is buying and selling way too frequently.
        #The behavior can be observed from the plot but no explanation
        else:
            if self.crossover<0:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                entry = self.dataclose
                self.order = self.sell_bracket(limitprice=entry*(1-(self.p.profit_percent/100)),
                                              stopprice=entry*(1+(self.p.loss_percent/100)),
                                              exectype=bt.Order.Market)

        

In [54]:
#A function for properly processing and printing analyzers
def printTradeAnalysis(analyzer):
    
    #Get the results we are interested in
    total_open = analyzer.total.open
    total_closed = analyzer.total.closed
    total_won = analyzer.won.total
    total_lost = analyzer.lost.total
    win_streak = analyzer.streak.won.longest
    lose_streak = analyzer.streak.lost.longest
    pnl_net = round(analyzer.pnl.net.total,2)
    strike_rate = (total_won / total_closed) * 100
    
    #Designate the rows
    h1 = ['Total Open', 'Total Closed', 'Total Won', 'Total Lost']
    h2 = ['Strike Rate','Win Streak', 'Losing Streak', 'PnL Net']
    r1 = [total_open, total_closed,total_won,total_lost]
    r2 = [strike_rate, win_streak, lose_streak, pnl_net]
    #Check which set of headers is the longest.
    if len(h1) > len(h2):
        header_length = len(h1)
    else:
        header_length = len(h2)
    #Print the rows
    
    print_list = [h1,r1,h2,r2]
    row_format ="{:<15}" * (header_length + 1)
    print("\n\nTrade Analysis Results:")
    for row in print_list:
        print(row_format.format('',*row))

In [58]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    cerebro.addstrategy(TestStrategy)

    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    #modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    #datapath = os.path.join(modpath, 'datas/orcl-1995-2014.txt')
    dirpath = os.path.dirname('/home/aviral/Desktop/Algo_trading_iitk/')
    datapath = os.path.join(dirpath, 'Nifty_2013.csv')

    #Check the format and positions of the datafeeds in case another file is used
    data = bt.feeds.GenericCSVData(
        dataname=datapath,
        
        #Even though the data file has 2013 in name, the data is actually of 2015
        fromdate=datetime.datetime(2015, 1, 1),
        todate=datetime.datetime(2015, 12, 31),
        nullvalue = 0.0,
        
        dtformat = ('%d/%m/%Y'),
        tmformat = ('%H:%M:%S'),
        
        timeframe = bt.TimeFrame.Minutes,
        compression = 1,
        #Compression is useful if the data is per minute but we want to use 5 min intervals
                
        datetime = 0,
        time = 1,
        open = 2,
        high = 3,
        low = 4,
        close = 5,
        volume = 6,
        openinterest = -1,
        reverse=False)

    cerebro.adddata(data)
    cerebro.broker.setcash(100000.0)

    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

    cerebro.broker.setcommission(commission=0.001)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    strats = cerebro.run()
    printTradeAnalysis(strats[0].analyzers.ta.get_analysis())

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
    #A weird error, scroll down and check. After september 8th all orders are displaying margin/cancelled


Starting Portfolio Value: 100000.00
2015-01-01, 10:38:00,  BUY CREATE, 8315.15
2015-01-01, 10:39:00,  BUY EXECUTED, Price: 8315.15, Cost: 83151.50, Comm 83.15
2015-01-01, 10:44:00,  SELL CREATE, 8313.90
2015-01-01, 10:45:00,  Order Margin/Rejected
2015-01-01, 10:45:00,  Order Canceled
2015-01-01, 10:45:00,  Order Canceled
2015-01-01, 12:19:00,  SELL CREATE, 8323.70
2015-01-01, 12:20:00,  Order Margin/Rejected
2015-01-01, 12:20:00,  Order Canceled
2015-01-01, 12:20:00,  Order Canceled
2015-01-01, 12:57:00,  SELL CREATE, 8319.50
2015-01-01, 12:58:00,  Order Margin/Rejected
2015-01-01, 12:58:00,  Order Canceled
2015-01-01, 12:58:00,  Order Canceled
2015-01-01, 13:00:00,  SELL CREATE, 8320.15
2015-01-01, 13:01:00,  Order Margin/Rejected
2015-01-01, 13:01:00,  Order Canceled
2015-01-01, 13:01:00,  Order Canceled
2015-01-01, 14:33:00,  SELL EXECUTED, Price: 8340.10, Cost: 83151.50, Comm 83.40
2015-01-01, 14:33:00,  Order Canceled
2015-01-01, 14:33:00,  OPERATION PROFIT, GROSS 249.45, NET 82.

2015-01-12, 15:01:00,  BUY CREATE, 8350.15
2015-01-12, 15:02:00,  BUY EXECUTED, Price: 8350.15, Cost: 83501.50, Comm 83.50
2015-01-13, 09:16:00,  SELL EXECUTED, Price: 8385.00, Cost: 83501.50, Comm 83.85
2015-01-13, 09:16:00,  Order Canceled
2015-01-13, 09:16:00,  OPERATION PROFIT, GROSS 348.50, NET 181.15
2015-01-13, 09:46:00,  BUY CREATE, 8360.35
2015-01-13, 09:47:00,  BUY EXECUTED, Price: 8360.55, Cost: 83605.50, Comm 83.61
2015-01-13, 10:20:00,  SELL CREATE, 8354.10
2015-01-13, 10:21:00,  Order Margin/Rejected
2015-01-13, 10:21:00,  Order Canceled
2015-01-13, 10:21:00,  Order Canceled
2015-01-13, 11:00:00,  SELL CREATE, 8357.15
2015-01-13, 11:01:00,  Order Margin/Rejected
2015-01-13, 11:01:00,  Order Canceled
2015-01-13, 11:01:00,  Order Canceled
2015-01-13, 12:28:00,  SELL CREATE, 8350.25
2015-01-13, 12:29:00,  Order Margin/Rejected
2015-01-13, 12:29:00,  Order Canceled
2015-01-13, 12:29:00,  Order Canceled
2015-01-13, 12:39:00,  SELL EXECUTED, Price: 8343.63, Cost: 83605.50, Comm

2015-01-22, 11:35:00,  BUY CREATE, 8775.00
2015-01-22, 11:36:00,  BUY EXECUTED, Price: 8775.00, Cost: 87750.00, Comm 87.75
2015-01-22, 12:17:00,  SELL CREATE, 8773.55
2015-01-22, 12:18:00,  Order Margin/Rejected
2015-01-22, 12:18:00,  Order Canceled
2015-01-22, 12:18:00,  Order Canceled
2015-01-22, 13:24:00,  SELL CREATE, 8766.85
2015-01-22, 13:25:00,  Order Margin/Rejected
2015-01-22, 13:25:00,  Order Canceled
2015-01-22, 13:25:00,  Order Canceled
2015-01-22, 13:57:00,  SELL CREATE, 8770.50
2015-01-22, 13:58:00,  Order Margin/Rejected
2015-01-22, 13:58:00,  Order Canceled
2015-01-22, 13:58:00,  Order Canceled
2015-01-22, 14:02:00,  SELL EXECUTED, Price: 8757.45, Cost: 87750.00, Comm 87.57
2015-01-22, 14:02:00,  Order Canceled
2015-01-22, 14:02:00,  OPERATION PROFIT, GROSS -175.50, NET -350.82
2015-01-22, 14:27:00,  BUY CREATE, 8770.25
2015-01-22, 14:28:00,  BUY EXECUTED, Price: 8770.25, Cost: 87702.50, Comm 87.70
2015-01-22, 14:51:00,  SELL CREATE, 8762.50
2015-01-22, 14:52:00,  Order

2015-02-03, 14:54:00,  BUY CREATE, 8835.75
2015-02-03, 14:55:00,  BUY EXECUTED, Price: 8835.75, Cost: 88357.50, Comm 88.36
2015-02-03, 15:09:00,  SELL CREATE, 8824.30
2015-02-03, 15:10:00,  Order Margin/Rejected
2015-02-03, 15:10:00,  Order Canceled
2015-02-03, 15:10:00,  Order Canceled
2015-02-03, 15:12:00,  SELL EXECUTED, Price: 8818.08, Cost: 88357.50, Comm 88.18
2015-02-03, 15:12:00,  Order Canceled
2015-02-03, 15:12:00,  OPERATION PROFIT, GROSS -176.72, NET -353.25
2015-02-04, 10:39:00,  BUY CREATE, 8807.00
2015-02-04, 10:40:00,  BUY EXECUTED, Price: 8807.00, Cost: 88070.00, Comm 88.07
2015-02-04, 10:41:00,  SELL CREATE, 8798.00
2015-02-04, 10:42:00,  Order Margin/Rejected
2015-02-04, 10:42:00,  Order Canceled
2015-02-04, 10:42:00,  Order Canceled
2015-02-04, 10:44:00,  SELL EXECUTED, Price: 8789.39, Cost: 88070.00, Comm 87.89
2015-02-04, 10:44:00,  Order Canceled
2015-02-04, 10:44:00,  OPERATION PROFIT, GROSS -176.14, NET -352.10
2015-02-04, 11:06:00,  BUY CREATE, 8802.00
2015-02

2015-02-12, 09:16:00,  SELL EXECUTED, Price: 8720.00, Cost: 86721.00, Comm 87.20
2015-02-12, 09:16:00,  Order Canceled
2015-02-12, 09:16:00,  OPERATION PROFIT, GROSS 479.00, NET 305.08
2015-02-12, 09:59:00,  BUY CREATE, 8678.60
2015-02-12, 10:00:00,  BUY EXECUTED, Price: 8678.60, Cost: 86786.00, Comm 86.79
2015-02-12, 10:10:00,  SELL CREATE, 8664.00
2015-02-12, 10:11:00,  Order Margin/Rejected
2015-02-12, 10:11:00,  Order Canceled
2015-02-12, 10:11:00,  Order Canceled
2015-02-12, 10:11:00,  SELL EXECUTED, Price: 8661.24, Cost: 86786.00, Comm 86.61
2015-02-12, 10:11:00,  Order Canceled
2015-02-12, 10:11:00,  OPERATION PROFIT, GROSS -173.57, NET -346.97
2015-02-12, 10:54:00,  BUY CREATE, 8663.90
2015-02-12, 10:55:00,  BUY EXECUTED, Price: 8663.90, Cost: 86639.00, Comm 86.64
2015-02-12, 11:16:00,  SELL CREATE, 8661.00
2015-02-12, 11:17:00,  Order Margin/Rejected
2015-02-12, 11:17:00,  Order Canceled
2015-02-12, 11:17:00,  Order Canceled
2015-02-12, 11:19:00,  SELL CREATE, 8661.10
2015-02-

...

2015-10-30, 15:04:00,  BUY CREATE, 8101.00
2015-10-30, 15:05:00,  Order Margin/Rejected
2015-10-30, 15:05:00,  Order Margin/Rejected
2015-10-30, 15:05:00,  Order Margin/Rejected
2015-11-02, 10:27:00,  BUY CREATE, 8061.30
2015-11-02, 10:28:00,  Order Margin/Rejected
2015-11-02, 10:28:00,  Order Margin/Rejected
2015-11-02, 10:28:00,  Order Margin/Rejected
2015-11-02, 11:11:00,  BUY CREATE, 8058.20
2015-11-02, 11:12:00,  Order Margin/Rejected
2015-11-02, 11:12:00,  Order Margin/Rejected
2015-11-02, 11:12:00,  Order Margin/Rejected
2015-11-02, 12:30:00,  BUY CREATE, 8040.00
2015-11-02, 12:31:00,  Order Margin/Rejected
2015-11-02, 12:31:00,  Order Margin/Rejected
2015-11-02, 12:31:00,  Order Margin/Rejected
2015-11-02, 12:35:00,  BUY CREATE, 8041.20
2015-11-02, 12:36:00,  Order Margin/Rejected
2015-11-02, 12:36:00,  Order Margin/Rejected
2015-11-02, 12:36:00,  Order Margin/Rejected
2015-11-02, 12:57:00,  BUY CREATE, 8043.00
2015-11-02, 12:58:00,  Order Margin/Rejected
2015-11-02, 12:58:00, 

2015-11-17, 10:20:00,  BUY CREATE, 7839.90
2015-11-17, 10:21:00,  Order Margin/Rejected
2015-11-17, 10:21:00,  Order Margin/Rejected
2015-11-17, 10:21:00,  Order Margin/Rejected
2015-11-17, 10:58:00,  BUY CREATE, 7849.90
2015-11-17, 10:59:00,  Order Margin/Rejected
2015-11-17, 10:59:00,  Order Margin/Rejected
2015-11-17, 10:59:00,  Order Margin/Rejected
2015-11-17, 11:48:00,  BUY CREATE, 7828.00
2015-11-17, 11:49:00,  Order Margin/Rejected
2015-11-17, 11:49:00,  Order Margin/Rejected
2015-11-17, 11:49:00,  Order Margin/Rejected
2015-11-17, 12:22:00,  BUY CREATE, 7826.30
2015-11-17, 12:23:00,  Order Margin/Rejected
2015-11-17, 12:23:00,  Order Margin/Rejected
2015-11-17, 12:23:00,  Order Margin/Rejected
2015-11-17, 13:09:00,  BUY CREATE, 7816.30
2015-11-17, 13:10:00,  Order Margin/Rejected
2015-11-17, 13:10:00,  Order Margin/Rejected
2015-11-17, 13:10:00,  Order Margin/Rejected
2015-11-17, 14:34:00,  BUY CREATE, 7858.75
2015-11-17, 14:35:00,  Order Margin/Rejected
2015-11-17, 14:35:00, 

2015-12-01, 10:19:00,  BUY CREATE, 7990.10
2015-12-01, 10:20:00,  Order Margin/Rejected
2015-12-01, 10:20:00,  Order Margin/Rejected
2015-12-01, 10:20:00,  Order Margin/Rejected
2015-12-01, 10:27:00,  BUY CREATE, 7989.00
2015-12-01, 10:28:00,  Order Margin/Rejected
2015-12-01, 10:28:00,  Order Margin/Rejected
2015-12-01, 10:28:00,  Order Margin/Rejected
2015-12-01, 10:38:00,  BUY CREATE, 7989.80
2015-12-01, 10:39:00,  Order Margin/Rejected
2015-12-01, 10:39:00,  Order Margin/Rejected
2015-12-01, 10:39:00,  Order Margin/Rejected
2015-12-01, 11:05:00,  BUY CREATE, 8001.85
2015-12-01, 11:06:00,  Order Margin/Rejected
2015-12-01, 11:06:00,  Order Margin/Rejected
2015-12-01, 11:06:00,  Order Margin/Rejected
2015-12-01, 11:34:00,  BUY CREATE, 7988.00
2015-12-01, 11:35:00,  Order Margin/Rejected
2015-12-01, 11:35:00,  Order Margin/Rejected
2015-12-01, 11:35:00,  Order Margin/Rejected
2015-12-01, 11:53:00,  BUY CREATE, 7988.65
2015-12-01, 11:54:00,  Order Margin/Rejected
2015-12-01, 11:54:00, 

2015-12-14, 09:53:00,  BUY CREATE, 7619.00
2015-12-14, 09:54:00,  Order Margin/Rejected
2015-12-14, 09:54:00,  Order Margin/Rejected
2015-12-14, 09:54:00,  Order Margin/Rejected
2015-12-14, 11:53:00,  BUY CREATE, 7657.15
2015-12-14, 11:54:00,  Order Margin/Rejected
2015-12-14, 11:54:00,  Order Margin/Rejected
2015-12-14, 11:54:00,  Order Margin/Rejected
2015-12-14, 12:34:00,  BUY CREATE, 7661.45
2015-12-14, 12:35:00,  Order Margin/Rejected
2015-12-14, 12:35:00,  Order Margin/Rejected
2015-12-14, 12:35:00,  Order Margin/Rejected
2015-12-14, 12:37:00,  BUY CREATE, 7662.00
2015-12-14, 12:38:00,  Order Margin/Rejected
2015-12-14, 12:38:00,  Order Margin/Rejected
2015-12-14, 12:38:00,  Order Margin/Rejected
2015-12-14, 13:55:00,  BUY CREATE, 7684.85
2015-12-14, 13:56:00,  Order Margin/Rejected
2015-12-14, 13:56:00,  Order Margin/Rejected
2015-12-14, 13:56:00,  Order Margin/Rejected
2015-12-14, 14:56:00,  BUY CREATE, 7651.70
2015-12-14, 14:57:00,  Order Margin/Rejected
2015-12-14, 14:57:00, 

2015-12-24, 10:13:00,  BUY CREATE, 7863.00
2015-12-24, 10:14:00,  Order Margin/Rejected
2015-12-24, 10:14:00,  Order Margin/Rejected
2015-12-24, 10:14:00,  Order Margin/Rejected
2015-12-24, 11:06:00,  BUY CREATE, 7864.80
2015-12-24, 11:07:00,  Order Margin/Rejected
2015-12-24, 11:07:00,  Order Margin/Rejected
2015-12-24, 11:07:00,  Order Margin/Rejected
2015-12-24, 11:42:00,  BUY CREATE, 7863.00
2015-12-24, 11:43:00,  Order Margin/Rejected
2015-12-24, 11:43:00,  Order Margin/Rejected
2015-12-24, 11:43:00,  Order Margin/Rejected
2015-12-24, 11:54:00,  BUY CREATE, 7862.35
2015-12-24, 11:55:00,  Order Margin/Rejected
2015-12-24, 11:55:00,  Order Margin/Rejected
2015-12-24, 11:55:00,  Order Margin/Rejected
2015-12-24, 12:05:00,  BUY CREATE, 7862.10
2015-12-24, 12:06:00,  Order Margin/Rejected
2015-12-24, 12:06:00,  Order Margin/Rejected
2015-12-24, 12:06:00,  Order Margin/Rejected
2015-12-24, 12:35:00,  BUY CREATE, 7861.80
2015-12-24, 12:36:00,  Order Margin/Rejected
2015-12-24, 12:36:00, 

In [57]:
cerebro.plot()
#Observe that a lot more green and red markers for buying and selling respectively
#Even at the points where there is no crossover whatsoever, and looks like the buy and sell signs are executed also

[[<Figure size 876x371 with 5 Axes>]]

In [32]:
#The analysing ratios and graphs
#Get the first strategy, in backtrader the analysing is done for each individual sttrategy
strat = strats[0]

#Pyfolio is the default name given by backtrader, check how to give a custom name
pyfoliozer = strat.analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()

pf.create_full_tear_sheet(
    returns,
    live_start_date=datetime.datetime(2000,1,3))


TypeError: an integer is required (got type str)